In [74]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import pandas as pd

In [75]:
raw_data = pd.read_csv('../datasets/raw/raw_salon_data.csv')
raw_data.drop(columns='Отметка времени', inplace=True)
synthetic_data = pd.read_csv('../datasets/synthetic/generated_synthetic_salon_data_852.csv')
df = pd.concat([raw_data, synthetic_data])
df.drop(columns=['Unnamed: 16', 'Unnamed: 17', 'Unnamed: 18'], inplace=True)

In [76]:
df.rename(columns={'Пожалуйста, выберите вариант стрижки, а ниже заполните анкету исходя из опыта - параметры клиента, которого вы стригли одним из приведенных способов':'Прическа'}, inplace=True)
df.drop_duplicates(inplace=True)
df.head()
#raw_data

,Прическа,Возраст,Форма Лица,Структура волос,Густота волос,Длина волос,Цвет волос,Среднее время на укладку(минут в день),Образ жизни,Стиль,Телосложение,Стиль одежды,Использование укладочных средств,Тип укладки,Использование фена
0,Андеркат,19-28,Прямоугольная,Прямые,Средняя,Средние,Шатен,3-5,Деловой,Смешанный,Спортивное,Спортивный,Да,Неряшливо-растрепанная,Да
1,Каскад,19-28,Круглая,Прямые,Средняя,Средние,Рыжий,25 минут,Спокойный,Минималист,Среднее,Casual,Да,Естественная,Да
2,Серферская укладка,29-45,Квадратная,Волнистые,Средняя,Средние,Шатен,30 минут,Творческий,Экспериментатор,Худощавое,Спортивный,Да,Неряшливо-растрепанная,Да
3,Каскад,46+,Ромбовидная,Прямые,Высокая,Вариант 3,Брюнет,30-40 минут,Деловой,Минималист,Среднее,Деловой,Нет,Четкая-монолитная,Да
4,Классика,29-45,Овальная,Волнистые,Средняя,Средние,Брюнет,20,Деловой,Минималист,Плотное,Деловой,Да,Естественная,Да


In [77]:
df.columns

Index(['Прическа', 'Возраст', 'Форма Лица', 'Структура волос', 'Густота волос',
       'Длина волос', 'Цвет волос', 'Среднее время на укладку(минут в день)',
       'Образ жизни', 'Стиль', 'Телосложение', 'Стиль одежды',
       'Использование укладочных средств', 'Тип укладки',
       'Использование фена'],
      dtype='object')

In [80]:
from typing import Union
import re
import numpy as np
import pandas as pd

def preprocessing(df):
    df = df[df['Цвет волос']!='Любой']
    df = df[df['Цвет волос']!='Любой ']
    df = df[df['Цвет волос']!='Other']
    df = df[df['Стиль одежды']!='Полу классика']
    df = df[df['Стиль одежды']!='Любой']

    def bucket_style_time(val: str) -> Union[str, float]:
        if pd.isna(val):
            return np.nan
        s = str(val).lower().strip()

        # «0 (не укладываете)»
        if re.search(r'\b0\b', s) and 'не' in s:
            return "0"

        # все числа в строке
        nums = [int(n) for n in re.findall(r'\d+', s)]
        if not nums:
            return np.nan

        t = sum(nums[:2]) / 2 if '-' in s else nums[0]

        if t == 0:
            return "0"
        elif t < 10:
            return "1-10"
        elif t < 30:
            return "10-30"
        else:
            return "30+"

    df['Среднее время на укладку(минут в день)'] = (
        df['Среднее время на укладку(минут в день)'].apply(bucket_style_time)
    )

    def normalize_style(val: Union[str, float]) -> Union[str, float]:
        """
        Приводит к четырём каноническим значениям:
        «Неряшливо-растрёпанная», «Естественная», «Чёткая-монолитная», «Other»
        """
        if pd.isna(val):
            return np.nan

        s = str(val).lower().replace('ё', 'е').strip()
        s = re.sub(r'\s+', ' ', s)          # много пробелов -> один
        s = re.sub(r'[–—−-]', '-', s)       # тире разных типов -> дефис

        if 'неряш' in s or 'растр' in s:
            return 'Неряшливо-растрёпанная'
        if 'естествен' in s:
            return 'Естественная'
        if 'четк' in s or 'монолит' in s or 'monolit' in s:
            return 'Чёткая-монолитная'
        return 'Other'

    df['Тип укладки'] = df['Тип укладки'].apply(normalize_style)

    return df

In [81]:
df = preprocessing(df)

In [83]:
df['Использование фена'].value_counts()

Использование фена
Да     660
Нет    313
Name: count, dtype: int64

In [33]:
from sklearn.model_selection import train_test_split
X = df.drop(columns='Прическа')
y = df['Прическа']
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.20, random_state=42, stratify=y, shuffle=True)